In [60]:
import torch
import matplotlib.pyplot as plt
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torchvision import transforms

In [125]:
transform = transforms.Compose([transforms.Resize((512,512)),transforms.ToTensor()])

In [126]:
training_data = datasets.Flowers102(root = 'data', split = 'train', download = True, transform = transform)
testing_data = datasets.Flowers102(root = 'data', split = 'test', download = True, transform = transform)

In [273]:
trainingLoader = DataLoader(training_data, batch_size=8, shuffle = True)
testingLoader = DataLoader(testing_data, batch_size=8, shuffle = True)

In [274]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,24,5),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.MaxPool2d(3,3)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(24, 72,5),
            nn.BatchNorm2d(72),
            nn.ReLU(),
            nn.MaxPool2d(3,3)
        )
        self.fc1 = nn.Linear(10368,5000)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(5000,1000)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(1000,102)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [275]:
network = NeuralNetwork()

In [276]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr = 0.005)

In [277]:
epoch = 100

for i in range(epoch):
    running_loss = 0
    for j, data in enumerate(trainingLoader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = network(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if j % 100 == 99:
            print(f'[{i + 1}, {j + 1:5d}] loss: {running_loss/100:.5f}')
            running_loss = 0

print('Finished Training')

[1,   100] loss: 4.62382
[2,   100] loss: 4.33543
[3,   100] loss: 3.96846
[4,   100] loss: 3.58053
[5,   100] loss: 3.17695
[6,   100] loss: 2.75202
[7,   100] loss: 2.31670
[8,   100] loss: 1.93557
[9,   100] loss: 1.57628
[10,   100] loss: 1.15128
[11,   100] loss: 0.83500
[12,   100] loss: 0.54071
[13,   100] loss: 0.34326
[14,   100] loss: 0.22295
[15,   100] loss: 0.15757
[16,   100] loss: 0.11124
[17,   100] loss: 0.08455
[18,   100] loss: 0.07093
[19,   100] loss: 0.05648
[20,   100] loss: 0.05069
[21,   100] loss: 0.04474
[22,   100] loss: 0.03671
[23,   100] loss: 0.03221
[24,   100] loss: 0.02798
[25,   100] loss: 0.02407
[26,   100] loss: 0.02425
[27,   100] loss: 0.02220
[28,   100] loss: 0.02037
[29,   100] loss: 0.01785
[30,   100] loss: 0.01635
[31,   100] loss: 0.01647
[32,   100] loss: 0.01484
[33,   100] loss: 0.01513
[34,   100] loss: 0.01384
[35,   100] loss: 0.01398
[36,   100] loss: 0.01217
[37,   100] loss: 0.01295
[38,   100] loss: 0.01088
[39,   100] loss: 0.0

In [278]:
correct = 0
total = 0

for data in testingLoader:
    images, labels = data
    output = network(images)
    _, predicted = torch.max(output.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
print(f"Network accuracy = {100 * (correct / total)}%")

Network accuracy = 25.223613595706617%
